# INPUT (Need to change before user run the script)

In [7]:
import pandas as pd
SKU_LIST = pd.read_excel('C:/Users/yamzh/Downloads/Question_1_dataset_Automation_Specialist.xlsx')
IMAGE_VIDEO_SAVE_PATH = 'C:/Users/yamzh/Downloads/ZALORA/'

# Library Import

In [1]:
!pip install pytube

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import urllib.request
import os
from IPython.display import clear_output
from pytube import YouTube

# Function

In [3]:

def get_product_html(DOMAIN_NAME,extension_link):
    # Get the PRODUCT_PAGE_HTML using SKU
    # *PRODUCT_PAGE_HTML is the page where we buy the product, eg. https://www.zalora.com.my/puma-asymmetric-women-s-top-black-2700404.html
    # *SEARCH_PAGE_HTML is the page where we search the product eg. https://www.zalora.com.my/catalog/?q=14C87AA11E1D1FGS
    search_page_html = scrape_link_html(DOMAIN_NAME,extension_link)
    product_extension_link = search_product_link(search_page_html)
    product_page_html = scrape_link_html(DOMAIN_NAME,product_extension_link)
    return product_page_html

    
def scrape_link_html(DOMAIN_NAME,extension_link):
    # Function for scrape the webpage in the HTML format
    HEADERS={'User-Agent': 'Mozilla/5.0'}
    response = requests.get(str(DOMAIN_NAME+extension_link), headers=HEADERS) 
    search_page_html = bs(response.text)
    return search_page_html


def search_product_link(bs_html):
    #Once we get the *SEARCH_PAGE_HTML using SKU, we will search the *PRODUCT_PAGE_LINK using this function
    find_product_link = str(bs_html.find_all('script',{ 'type' : 'text/javascript' }))
    product_link = find_product_link[find_product_link.index('"link"'):].split(",")[0][8:-1]
    return str(product_link)



def get_product_image_webpage_list(product_html):
    #Get the IMAGE_LIST using PRODUCT_PAGE_HTML
    return product_html.find_all('meta',{'property':"og:image"})


def search_product_youtube_link(product_html):
    #Get the YoutubeLink using PRODUCT_PAGE_HTML
    video_html = product_html.find_all('div',{'id':"videoContainer"})
    if video_html == []:
        return
    youtube_id = bs(str(video_html), 'html.parser').div.find("input")["value"]
    if youtube_id == '-':
        return
    return str("https://www.youtube.com/watch?v=" + youtube_id)                                         
                                         
    
def save_image_list_to_path(IMAGE_SAVE_PATH,sku_id,product_image_list):
    #Downalod the image to the path we want
    path = str(IMAGE_SAVE_PATH + sku_id)
    if not os.path.exists(path):
        os.makedirs(path)
    for index in range(0,len(product_image_list)):
        file_name_and_location = os.path.join(path, str(sku_id)+"_"+str(index)+".jpg")
        urllib.request.urlretrieve(product_image_list[index]['content'], file_name_and_location)
        
        
def download_youtube_to_path(IMAGE_VIDEO_SAVE_PATH,sku_id,product_youtube_link):
    #Downalod the video to the path we want
    if product_youtube_link == None:
        return
    yt = YouTube(product_youtube_link)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    path = str(IMAGE_VIDEO_SAVE_PATH + sku_id)
    if not os.path.exists(path):
        os.makedirs(path)
    yt.download(path)

# Program Run

In [6]:
#FIX VARIABLE
DOMAIN_NAME = 'https://www.zalora.com.my/'
SKU_LIST["error_code"] = None
SKU_LIST["completed"] = 0

for index, sku_id in enumerate(SKU_LIST.Sku):
    try:
        """LOADING SCREEN"""
        clear_output(wait=True)
        print(str(index+1) + "/" + str(len(SKU_LIST)))
        print(sku_id)

        """WEB SCRAPE BACKEND PROCESS"""
        sku_extension_link = 'catalog/?q=' + sku_id
        product_html = get_product_html(DOMAIN_NAME, sku_extension_link)                    #Get the Specific Product Page using the SKU id
                                                                                            #Assume that SKU and PRODUCT is 1:1 relationship
            
            

        product_image_list = get_product_image_webpage_list(product_html)                   #Get the Image List in the PRODUCT Page 
        product_youtube_link = search_product_youtube_link(product_html)                    #Get the YouTube Link in the PRODUCT Page
        
    

        save_image_list_to_path(IMAGE_VIDEO_SAVE_PATH,sku_id,product_image_list)            #Download all the image in Image List
        download_youtube_to_path(IMAGE_VIDEO_SAVE_PATH,sku_id,product_youtube_link)         #Download the video in Youtube link

        SKU_LIST.at[index, 'completed'] = 1                                                 #Mark "completed" in SKU_LIST



    except Exception as e:
        SKU_LIST.at[index, 'error_code'] = e                                               #Mark error if occur
        
    finally:
        SKU_LIST.to_csv("C:/Users/yamzh/Downloads/ZALORA/output.csv")

    

380/380
9AFFEAA38B57A7GS
